In [4]:
import pandas as pd
import numpy as np
import csv
#import nltk
from sklearn.model_selection import cross_val_score
#from nltk.corpus import movie_reviews
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline

In [5]:
data = pd.read_csv('products_sentiment_sample_submission.csv')

In [6]:
file = open('products_sentiment_train.tsv')

In [7]:
X=[]
y=[]
with open('products_sentiment_train.tsv') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        X.append(row[0])
        y.append(row[1])
        
        
X_test=[]
with open('products_sentiment_test.tsv') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        X_test.append(row[1])
X_test=X_test[1:]

In [5]:
#ищем лучший векторайзер и классифаер
for vec in [CountVectorizer, TfidfVectorizer]:
    for clf in [LinearSVC, SGDClassifier, LogisticRegression]:
        ppl = Pipeline([
                ('vectorizer', vec(ngram_range=(1,5), token_pattern=r'\b[^\d\W]+\b', analyzer = 'word')),
                ('classifier', clf())
            ])

#ppl.fit(X, y)
        print((cross_val_score(ppl, X, y, cv=10).mean()))

0.7675078876971925


KeyboardInterrupt: 

In [ ]:
#ищем лучший n-граммы c TfidfVectorizer и SGDClassifier
ng=[]
for i in range(1,5):
    for j in range(1,5):
        if j>i:
            ppl = Pipeline([
                    ('vectorizer', TfidfVectorizer(ngram_range=(i,j))),
                    ('classifier', SGDClassifier())
                ])
            ng.append([(cross_val_score(ppl, X, y, cv=5).mean()), i , j])
    #ppl.fit(X, y)
        

In [ ]:
ng.sort(key=lambda x: x[0])
ng[-1]

In [8]:
#игнорируем слова, у которых частота больше 300
ppl = Pipeline([
        ('vectorizer', TfidfVectorizer(ngram_range=(1,4), max_df=300)),
        ('classifier', SGDClassifier())
    ])
ppl.fit(X, y)
print([(cross_val_score(ppl, X, y, cv=10).mean())])

[0.787980774519363]


In [14]:
clf = SGDClassifier(loss='log')
vect = TfidfVectorizer(ngram_range=(1,4), max_df=300)

clf.fit(vect.fit_transform(X), y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [24]:
vectorized = vect.transform(['i really liked that movie'])
(clf.predict(vectorized)[0], clf.predict_proba(vectorized)[0].max())

('1', 0.7232779351870396)

In [16]:
import pickle
with open('clf.pkl', 'wb') as f:
    pickle.dump(clf, f)
with open('vect.pkl', 'wb') as f:
    pickle.dump(vect, f)
    
    
    self.get_probability_words(prediction_probability) + " " + self.classes_dict[class_prediction]

In [ ]:
#попытки улучшить результат добавлением стоп-слов и/или определением минимальной частоты слов не увенчались успехом
ppl = Pipeline([
        ('vectorizer', TfidfVectorizer(ngram_range=(1,4), min_df=10, max_df=300)),
        ('classifier', SGDClassifier())
    ])
print([(cross_val_score(ppl, X, y, cv=10).mean())])

ppl = Pipeline([
        ('vectorizer', TfidfVectorizer(ngram_range=(1,4),min_df=5, max_df=300)),
        ('classifier', SGDClassifier())
    ])
ppl.fit(X, y)
print([(cross_val_score(ppl, X, y, cv=10).mean())])

ppl = Pipeline([
        ('vectorizer', TfidfVectorizer(ngram_range=(1,4), min_df=5, stop_words='english', max_df=300)),
        ('classifier', SGDClassifier())
    ])
ppl.fit(X, y)
print([(cross_val_score(ppl, X, y, cv=10).mean())])

ppl = Pipeline([
        ('vectorizer', TfidfVectorizer(ngram_range=(1,4), stop_words='english', max_df=300)),
        ('classifier', SGDClassifier())
    ])
ppl.fit(X, y)
print([(cross_val_score(ppl, X, y, cv=10).mean())])

In [ ]:
data.y = ppl.predict(X_test)

In [ ]:
data.to_csv('try1.csv', index=False)